# Orchestration Service Tutorial

This notebook demonstrates how to use the SDK to interact with the Orchestration Service, enabling you to build AI-driven workflows by combining multiple modules such as templating, large language models (LLM), and content filtering.

## Initializing the Orchestration Service
Before using the SDK, you need to set up a virtual deployment of the Orchestration Service. Once deployed, you'll have access to a unique endpoint URL.

In [1]:
YOUR_API_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d7af2243a66e5000"

## Basic Orchestration Pipeline

Now that you have YOUR_API_URL, let's walk through a basic orchestration pipeline for a translation task.

### Step 1: Define the Template and Default Input Values

In [2]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}"
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="German"),
    ],
)

### Step 2: Define the LLM

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 100, "temperature": 0})

### Step 3: Create the Orchestration Configuration

In [4]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

### Step 4: Run the Orchestration Request

In [5]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)
result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])

In [6]:
print(result.orchestration_result.choices[0].message.content)

Der Orchestrierungsdienst funktioniert!


## Optional Modules

### Content Filtering

The Orchestration Service supports content filtering to moderate input and output. Let's explore how to set up and use content filtering.

In [8]:
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter

content_filter = AzureContentFilter(
    hate=0,
    sexual=4,
    self_harm=0,
    violence=2,
)

orchestration_service.config.input_filters = [content_filter]
orchestration_service.config.output_filters = [content_filter]

In [9]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(template_values=[
        TemplateValue(name="text", value="I hate you")
    ])
except OrchestrationError as error:
    print(error.message)

Content filtered due to Safety violations. Please modify the prompt and try again.


## Advanced Examples

In [10]:
service = OrchestrationService(api_url=YOUR_API_URL)

### Translation Service

In [11]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class TranslationService:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful translation assistant."),
                    UserMessage(
                        "Translate the following text to {{?to_lang}}: {{?text}}"
                    ),
                ],
                defaults=[
                    TemplateValue(name="to_lang", value="English"),
                ],
            ),
            llm=LLM(name="gpt-4o"),
        )

    def translate(self, text, to_lang):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="to_lang", value=to_lang),
                TemplateValue(name="text", value=text),
            ],
        )

        return response.orchestration_result.choices[0].message.content


In [12]:
translator = TranslationService(orchestration_service=service)

In [13]:
result = translator.translate(text="Hello, world!", to_lang="French")
print(result)

Bonjour, le monde!


In [14]:
result = translator.translate(text="Hello, world!", to_lang="Spanish")
print(result)

¡Hola, mundo!


In [15]:
result = translator.translate(text="Hello, world!", to_lang="German")
print(result)

Hallo, Welt!


### Chatbot with Memory

In [16]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-4"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )
        
        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

In [17]:
bot = ChatBot(orchestration_service=service)

In [18]:
print(bot.chat("Hello, how are you?"))

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. What can I assist you with today?


In [19]:
print(bot.chat("What's the weather like today?"))

I don't have real-time data capabilities, so I can't provide current weather information. I recommend checking a reliable weather website or using a weather app on your mobile device to get the most accurate and up-to-date information. How else may I assist you today?


In [20]:
print(bot.chat("Can you remember what I first asked you?"))

Yes, you first asked "Hello, how are you?" How can I assist you further today?


### Sentiment Analysis with Few Shot Learning 

In [21]:
from typing import List, Tuple

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import (
    SystemMessage,
    UserMessage,
    AssistantMessage,
)
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

class FewShotLearner:
    def __init__(
        self,
        orchestration_service: OrchestrationService,
        system_message: SystemMessage,
        examples: List[Tuple[UserMessage, AssistantMessage]],
    ):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=self._create_few_shot_template(system_message, examples),
            llm=LLM(name="gpt-4o"),
        )

    @staticmethod
    def _create_few_shot_template(
        system_message: SystemMessage,
        examples: List[Tuple[UserMessage, AssistantMessage]],
    ) -> Template:
        messages = [system_message]

        for example in examples:
            messages.append(example[0])
            messages.append(example[1])
        messages.append(UserMessage("{{?user_input}}"))

        return Template(messages=messages)

    def predict(self, user_input: str) -> str:
        response = self.service.run(
            config=self.config,
            template_values=[TemplateValue(name="user_input", value=user_input)],
        )

        return response.orchestration_result.choices[0].message.content

In [22]:
sentiment_examples = [
    (UserMessage("I love this product!"), AssistantMessage("Positive")),
    (UserMessage("This is terrible service."), AssistantMessage("Negative")),
    (UserMessage("The weather is okay today."), AssistantMessage("Neutral")),
]

In [23]:
sentiment_analyzer = FewShotLearner(
    orchestration_service=service,
    system_message=SystemMessage(
        "You are a sentiment analysis assistant. Classify the sentiment as Positive, Negative, or Neutral."
    ),
    examples=sentiment_examples,
)

In [24]:
print(sentiment_analyzer.predict("The movie was a complete waste of time!"))

Negative


In [25]:
print(
    sentiment_analyzer.predict("The traffic was fortunately unusually light today.")
)

Positive


In [26]:
print(
    sentiment_analyzer.predict("I'm not sure how I feel about the recent events.")
)

Neutral


In [27]:
print(
    sentiment_analyzer.predict("The movie was so bad that I couldn't stop watching")
)

Negative
